In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.simplefilter("ignore")

In [3]:
df=pd.read_csv("CleanedLoanData.csv")

In [5]:
df.head()

,person_age,person_education,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status,person_gender_male,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,previous_loan_defaults_on_file_Yes
0,0.400888,3,7.116215,0,44.928204,16.02,0.181480,3.0,561,1,False,False,False,True,False,False,False,True,False,False
1,0.400869,2,6.401416,0,17.192952,11.14,0.064705,2.0,504,0,False,False,True,False,True,False,False,False,False,True
2,0.400930,2,6.406924,3,27.675832,12.87,0.175681,3.0,635,1,False,False,False,False,False,False,True,False,False,False
3,0.400904,0,7.154530,0,44.928204,15.23,0.175681,2.0,675,1,False,False,False,True,False,False,True,False,False,False
4,0.400918,3,7.084614,1,44.928204,14.27,0.185389,4.0,586,1,True,False,False,True,False,False,True,False,False,False


In [7]:
x=df.drop("loan_status",axis=1)
y=df["loan_status"]

In [9]:
contnious=["person_age","person_income","loan_amnt","loan_int_rate","loan_percent_income",
          "credit_score"]

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=77)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train[contnious]=sc.fit_transform(x_train[contnious])
x_test[contnious]=sc.transform(x_test[contnious])

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
estimator=RandomForestClassifier(random_state=True)
param_grid={"n_estimators":list(range(101))}
grid=GridSearchCV(estimator,param_grid,cv=5,scoring="accuracy")
grid.fit(x_train,y_train)
grid.best_params_

{'n_estimators': 99}

In [31]:
impDF=pd.DataFrame(grid.best_estimator_.feature_importances_,
                  columns=["Feature_Value"],
                  index=x_train.columns.tolist()
                )
impDF
impCol=impDF.index.tolist()


In [35]:
x_train=x_train[impCol]
x_test=x_test[impCol]

In [37]:
#Modelling
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=45,n_estimators=99)
model.fit(x_train,y_train)

#Prediction
ypredTrain=model.predict(x_train)
ypredTest=model.predict(x_test)

#Accuracy
from sklearn.metrics import accuracy_score
TrainAccuracy=accuracy_score(y_train,ypredTrain)
TestAccuracy=accuracy_score(y_test,ypredTest)

#CrossVal
from sklearn.model_selection import cross_val_score
TrainCV=cross_val_score(model,x_train,y_train,cv=5,scoring="accuracy").mean()

#print
print("TrainAccuracy: ",TrainAccuracy," ","TestAccuracy: ",TestAccuracy," ","TrainCV",TrainCV)

TrainAccuracy:  1.0   TestAccuracy:  0.9265473941549061   TrainCV 0.92815868330478
